# Segmenting and Clustering Neighbourhoods in Toronto - Part 1

Importing libraries 

In [ ]:
import pandas as pd
import numpy as np
import requests

Get the HTML page of Wiki, and using read_html we convert the html data into list of Data frame objects.

Remove cells which have borrow not assigned.


In [ ]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki)

wiki_raw = pd.read_html(wiki_page.content, header = 0)[0]
df = wiki_raw[wiki_raw.Neighborhood != 'Not assigned']
df.reset_index(inplace = True)
df.head()

,index,Postal Code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
df.groupby(['Postal Code']).first()

,index,Borough,Neighborhood
Postal Code,,,
M1B,9,Scarborough,"Malvern, Rouge"
M1C,18,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,27,Scarborough,"Guildwood, Morningside, West Hill"
M1G,36,Scarborough,Woburn
M1H,45,Scarborough,Cedarbrae
...,...,...,...
M9N,98,York,Weston
M9P,107,Etobicoke,Westmount
M9R,116,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


The new Wiki link already has nieghborhood merged according to Postal Code and Borough


In [ ]:
len(df['Postal Code'].unique())

103

In [ ]:
df[df['Borough'] == 'Not assigned']

,index,Postal Code,Borough,Neighborhood


In [ ]:
df.shape

(103, 4)

# Part 2

In [ ]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


Import geocoder lib to fetch latitude and longitude

In [ ]:
import geocoder

geocoder wasn't working that's why using csv file given in the course

In [ ]:
url = 'http://cocl.us/Geospatial_data'

In [ ]:
df_geo = pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Checking dtypes of our columns

In [ ]:
df_geo.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [ ]:
df.dtypes

index            int64
Postal Code     object
Borough         object
Neighborhood    object
dtype: object

Shapes of both our Dataframes

In [ ]:
df.shape

(103, 4)

In [ ]:
df_geo.shape

(103, 3)

Joining both df's and doing some cleaning

In [ ]:
df = df.join(df_geo.set_index('Postal Code'), on='Postal Code')
df

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [ ]:
df = df.reset_index()

In [ ]:
df.drop(['index'], axis = 'columns', inplace = True)

In [ ]:
df = df.set_index('level_0')

In [ ]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
level_0,,,,,
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
df = df.rename(index = {'level_0' : 'index'})

In [ ]:
df.index.name = 'index'

In [ ]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
index,,,,,
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
df.shape

(103, 5)